In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import psutil
import subprocess

def kill_port(port: int):
    for proc in psutil.process_iter():
        try:
            for con in proc.net_connections(kind="inet"):
                if con.laddr.port == port:
                    proc.kill()
        except (psutil.AccessDenied, psutil.NoSuchProcess):
            pass

kill_port(8000)
process = subprocess.Popen(["python", "server.py"])
print(f"Started server.py with PID {process.pid}")


In [ ]:
import uuid

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph_bigtool import create_agent

from langgraph.store.memory import InMemoryStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from typing_extensions import Annotated


llm = ChatOpenAI(model="gpt-4o-mini")

# 2) Server-Konfiguration definieren
servers_config = {
    "my_mcp_server": {
        "transport": "sse",
        "url": "http://127.0.0.1:8000/sse",
    },
}

client = MultiServerMCPClient(servers_config)
await client.__aenter__()
server_tools = client.get_tools()


In [ ]:
server_tools

In [ ]:
tool_registry = {
    str(uuid.uuid4()): tool
    for tool in server_tools
}
tool_registry

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
        "fields": ["description"],
    }
)

In [ ]:
for tool_id, tool in tool_registry.items():
    store.put(
        ("tools",),
        tool_id,
        {
            "description": f"{tool.name}: {tool.description}"
        },
    )

In [ ]:
def retrieve_tools(
    query: str,
    *,
    store: Annotated[BaseStore, InjectedStore],
) -> list[str]:
    """
    Perform a semantic search in the store for tools matching the given query.
    Returns a list of tool IDs that are likely relevant.
    """
    results = store.search(("tools",), query=query, limit=3)
    tool_ids = [result.key for result in results]
    return tool_ids

In [ ]:
builder = create_agent(
    llm=llm,
    tool_registry=tool_registry,
    retrieve_tools_function=retrieve_tools
)
agent = builder.compile(store=store)

query = "Please add 3 and 7. Use any available tools to do so."
result = await agent.ainvoke({"messages": query})

print("## Result ##")
for msg in result["messages"]:
    print(msg.content)
